# Случайные леса

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с бустингом, предоставляемым библиотекой `CatBoost`.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека.

In [100]:
import inspect
import random
from collections import Counter
from dataclasses import dataclass
from itertools import product
from typing import Callable, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import pandas
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [101]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)


# Этой функцией будут помечены все места, которые необходимо дозаполнить
# Это могут быть как целые функции, так и отдельные части внутри них
# Всегда можно воспользоваться интроспекцией и найти места использования этой функции :)
def todo():
    stack = inspect.stack()
    caller_frame = stack[1]
    function_name = caller_frame.function
    line_number = caller_frame.lineno
    raise NotImplementedError(f"TODO at {function_name}, line {line_number}")


SEED = 0xC0FFEE
set_seed(SEED)

In [102]:
def mode(data):
    counts = Counter(data)
    return counts.most_common(n=1)[0][0]

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bootstrap. Элементы, которые не вошли в новую обучающую выборку, образуют **out-of-bag** выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bootstrap. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [103]:
# Для начала реализуем сами критерии


def gini(x: npt.ArrayLike) -> float:
    """
    Calculate the Gini impurity of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Gini impurity value.
    """
    _, cnt_labels = np.unique(x, return_counts=True)
    p_labels = cnt_labels / x.size
    return np.sum(p_labels * (1 - p_labels))


def entropy(x: npt.ArrayLike) -> float:
    """
    Calculate the entropy of a list or array of class labels.

    Args:
        x (ArrayLike): Array-like object containing class labels.

    Returns:
        float: Entropy value.
    """
    _, cnt_labels = np.unique(x, return_counts=True)
    p_labels = cnt_labels / x.size
    return -np.sum(p_labels * np.log(p_labels))


def gain(left_y: npt.ArrayLike, right_y: npt.ArrayLike, criterion: Callable[[npt.ArrayLike], float]) -> float:
    """
    Calculate the information gain of a split using a specified criterion.

    Args:
        left_y (ArrayLike): Class labels for the left split.
        right_y (ArrayLike): Class labels for the right split.
        criterion (Callable): Function to calculate impurity (e.g., gini or entropy).

    Returns:
        float: Information gain from the split.
    """
    parent_size = left_y.size + right_y.size

    parent_impurity = criterion(np.hstack((left_y, right_y)))
    left_impurity = criterion(left_y)
    right_impurity = criterion(right_y)

    return parent_impurity - left_impurity * (left_y.size / parent_size) - right_impurity * (right_y.size / parent_size)

In [104]:
@dataclass
class DecisionTreeLeaf:
    classes: np.ndarray

    def __post_init__(self):
        self.max_class = mode(self.classes)


@dataclass
class DecisionTreeInternalNode:
    split_dim: int
    left: Union["DecisionTreeInternalNode", DecisionTreeLeaf]
    right: Union["DecisionTreeInternalNode", DecisionTreeLeaf]


DecisionTreeNode = Union[DecisionTreeInternalNode, DecisionTreeLeaf]

In [105]:
from typing import Optional

In [106]:
class DecisionTree:
    criterions = {"gini": gini, "entropy": entropy}

    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self._max_features = np.sqrt(X.shape[1]).astype(int) if max_features == "auto" else max_features
        self._max_depth = max_depth
        self._min_samples_leaf = min_samples_leaf
        self._criterion = self.criterions[criterion]

        bootstrap_indices = np.random.choice(np.arange(y.size), size=y.size, replace=True)
        self._X = X[bootstrap_indices]
        self._y = y[bootstrap_indices]

        mask = np.zeros(y.size, dtype=bool)
        mask[bootstrap_indices] = True
        self._out_of_bag_X = X[~mask]
        self._out_of_bag_y = y[~mask]
        self._root = None
        self._is_fit = False

    @property
    def out_of_bag(self) -> Tuple[np.ndarray, np.ndarray]:
        return self._out_of_bag_X, self._out_of_bag_y

    def _get_best_information_gain_dim(
        self, points: np.ndarray, classes: np.ndarray, feature_indices: np.ndarray
    ) -> Optional[int]:
        best_dim = None
        best_information_gain = 0
        for dim in feature_indices:
            left_node = classes[points[:, dim] == 0]
            right_node = classes[points[:, dim] == 1]
            if left_node.size >= self._min_samples_leaf and right_node.size >= self._min_samples_leaf:
                curr_gain = gain(left_node, right_node, self._criterion)
                if curr_gain > best_information_gain:
                    best_information_gain = curr_gain
                    best_dim = dim
        return best_dim

    def _build_node(self, points: np.ndarray, classes: np.ndarray, depth: int) -> DecisionTreeNode:
        if np.all(classes == classes[0]):
            return DecisionTreeLeaf(classes)
        if self._max_depth is not None and depth == self._max_depth:
            return DecisionTreeLeaf(classes)

        feature_indices = np.random.randint(0, self._X.shape[1], size=self._max_features)
        best_dim = self._get_best_information_gain_dim(points, classes, feature_indices)
        if best_dim is None:
            return DecisionTreeLeaf(classes)

        left_indices = points[:, best_dim] == 0
        left_node = self._build_node(points[left_indices], classes[left_indices], depth + 1)
        right_node = self._build_node(points[~left_indices], classes[~left_indices], depth + 1)
        return DecisionTreeInternalNode(best_dim, left_node, right_node)

    def _predict(self, points: np.ndarray, node: DecisionTreeNode) -> np.ndarray:
        if not self._is_fit:
            raise ValueError("DecisionTree is not fit")

        def process_node(indices, curr_node):
            if isinstance(curr_node, DecisionTreeLeaf):
                predictions[indices] = curr_node.max_class
                return

            split_dim = curr_node.split_dim

            left_mask = points[indices, split_dim] == 0
            left_indices = indices[left_mask]
            right_indices = indices[~left_mask]

            if left_indices.size > 0:
                process_node(left_indices, curr_node.left)
            if right_indices.size > 0:
                process_node(right_indices, curr_node.right)

        predictions = np.empty(points.shape[0], dtype=self._y.dtype)
        indices = np.arange(points.shape[0])
        process_node(indices, node)

        return predictions

    def fit(self):
        self._root = self._build_node(self._X, self._y, 0)
        self._is_fit = True

    def predict(self, points: np.ndarray) -> np.ndarray:
        return self._predict(points, self._root)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [107]:
from sklearn.base import BaseEstimator, ClassifierMixin

In [108]:
class RandomForestClassifier(BaseEstimator, ClassifierMixin):

    _n_features: int = None

    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self._estimators = []

    @property
    def estimators(self) -> List[DecisionTree]:
        return self._estimators

    @property
    def n_features(self) -> int:
        if self._n_features is None:
            raise RuntimeError("Fit random forest before accessing to number of features properties")
        return self._n_features

    def fit(self, X, y):
        RandomForestClassifier._n_features = X.shape[1]
        for i in range(self.n_estimators):
            decision_tree = DecisionTree(
                X,
                y,
                self.criterion,
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                max_features=self.max_features,
            )
            decision_tree.fit()
            self.estimators.append(decision_tree)

    def predict(self, X):
        all_predicts = np.array([estimator.predict(X) for estimator in self._estimators])
        modes = []
        for col in all_predicts.T:
            values, counts = np.unique(col, return_counts=True)
            mode = values[np.argmax(counts)]
            modes.append(mode)
        return np.array(modes)

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [109]:
def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)
    return np.mean(y_true == y_pred)


def feature_importance(rfc):
    importance = np.zeros(rfc.n_features)
    n_estimators = len(rfc.estimators)
    for estimator in rfc.estimators:
        X_oob, y_oob = estimator.out_of_bag
        oob_pred = estimator.predict(X_oob)
        err_oob = 1 - accuracy_score(y_oob, oob_pred)
        for i in range(rfc.n_features):
            X_oob_perm = X_oob.copy()
            np.random.shuffle(X_oob_perm[:, i])
            oob_pred_j = estimator.predict(X_oob_perm)
            err_oob_j = 1 - accuracy_score(y_oob, oob_pred_j)
            importance[i] += err_oob_j - err_oob
    return importance / n_estimators


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [110]:
def synthetic_dataset(size):
    X = [
        (np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, i % 6 == 0, i % 3 == 2, np.random.randint(0, 2))
        for i in range(size)
    ]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)


X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
rfc.predict(X)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [ 1.70452947e-03 -2.44826556e-04  1.54450440e-01  1.43690023e-01
  2.94225041e-01  1.47239289e-03]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [111]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]

    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [112]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

In [113]:
X_train_unique, idx = np.unique(X_train, axis=0, return_index=True)
y_age_train_unique = y_age_train[idx]
y_sex_train_unique = y_sex_train[idx]

In [114]:
train_size = X_train_unique.shape[0]
ratio = np.mean(X_train, axis=0)
mask = (ratio < 0.03) | (ratio > 0.97)
X_train_selected = X_train_unique[:, ~mask]
X_test_selected = X_test[:, ~mask]
print(f"Removed {np.sum(mask)} features with numbers {np.where(mask)[0]}")

Removed 11 features with numbers [ 32  33  35  40  55  62  70  73  97 124 148]


In [115]:
del_idx = np.where(mask)[0]
new_features = []
for i in range(len(features)):
    if i not in del_idx:
        new_features.append(features[i])

In [116]:
sub_train_cnt = np.sum(X_train_selected, axis=1).reshape(-1, 1)
sub_test_cnt = np.sum(X_test_selected, axis=1).reshape(-1, 1)
X_train_with_subs = np.hstack((X_train_selected, sub_train_cnt))
X_test_with_subs = np.hstack((X_test_selected, sub_test_cnt))
new_features.append("subscription_cnt")

#### Возраст

In [117]:
from sklearn.model_selection import GridSearchCV

In [118]:
rfc = RandomForestClassifier()
rfc_param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 5, 10, 15, 25],
    "min_samples_leaf": [1, 10, 20, 50],
    "n_estimators": [10, 50, 100],
}
grid_search = GridSearchCV(estimator=rfc, param_grid=rfc_param_grid, scoring="accuracy", cv=3, verbose=1)

grid_search.fit(X_train_with_subs, y_age_train_unique)
print("Лучшие гиперпараметры:", grid_search.best_params_)

print("Лучший Accuracy:", grid_search.best_score_)

Fitting 3 folds for each of 120 candidates, totalling 360 fits
Лучшие гиперпараметры: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 100}
Лучший Accuracy: 0.7079068387479602


In [119]:
rfc = RandomForestClassifier(**grid_search.best_params_)

rfc.fit(X_train_with_subs, y_age_train_unique)
print("Accuracy:", np.mean(rfc.predict(X_test_with_subs) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), new_features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.7351828499369483
Most important features:
1. ovsyanochan
2. 4ch
3. styd.pozor
4. rhymes
5. mudakoff
6. dayvinchik
7. pravdashowtop
8. rapnewrap
9. iwantyou
10. bot_maxim
11. tumblr_vacuum
12. reflexia_our_feelings
13. pixel_stickers
14. ohhluul
15. bog_memes
16. leprum
17. xfilm
18. ne1party
19. i_d_t
20. iloveu01


#### Пол

In [120]:
rfc = RandomForestClassifier()
rfc_param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 5, 10, 15, 25],
    "min_samples_leaf": [1, 10, 20, 50],
    "n_estimators": [10, 50, 100],
}
grid_search = GridSearchCV(estimator=rfc, param_grid=rfc_param_grid, scoring="accuracy", cv=3, verbose=1)

grid_search.fit(X_train_with_subs, y_sex_train_unique)
print("Лучшие гиперпараметры:", grid_search.best_params_)

print("Лучший Accuracy:", grid_search.best_score_)

Fitting 3 folds for each of 120 candidates, totalling 360 fits
Лучшие гиперпараметры: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 50}
Лучший Accuracy: 0.8494288681204569


In [121]:
rfc = RandomForestClassifier(**grid_search.best_params_)
rfc.fit(X_train_with_subs, y_sex_train_unique)
print("Accuracy:", np.mean(rfc.predict(X_test_with_subs) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8650693568726355
Most important features:
1. 40kg
2. girlmeme
3. pho
4. club128730587
5. public_of_music
6. webestano
7. gm.power
8. academyofman
9. 9o_6o_9o
10. iwantyou
11. cook_bon
12. iloveu01
13. zerofat
14. cook_good
15. thesmolny
16. face
17. sketch.books
18. leprum
19. trahninormalnost1
20. rapnewrap


### CatBoost
В качестве аьтернативы попробуем CatBoost.

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [122]:
X, y = synthetic_dataset(1000)

cb_model = CatBoostClassifier(loss_function="MultiClass", verbose=0)
cb_model.fit(X, y)
y_pred = cb_model.predict(X)

print("Accuracy:", accuracy_score(y_pred, y))
print("Importance:", cb_model.feature_importances_)

Accuracy: 1.0
Importance: [9.45918392e-03 4.77433498e-03 2.78335273e+01 2.78550156e+01
 4.42922373e+01 4.98630028e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [123]:
from sklearn.metrics import accuracy_score

In [124]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(
    X_train, y_age_train, y_sex_train, train_size=0.8
)

In [125]:
X_train_unique, idx = np.unique(X_train, axis=0, return_index=True)
y_age_train_unique = y_age_train[idx]
y_sex_train_unique = y_sex_train[idx]

In [126]:
train_size = X_train_unique.shape[0]
ratio = np.mean(X_train, axis=0)
mask = (ratio < 0.03) | (ratio > 0.97)
X_train_selected = X_train_unique[:, ~mask]
X_eval_selected = X_eval[:, ~mask]
X_test_selected = X_test[:, ~mask]
print(f"Removed {np.sum(mask)} features with numbers {np.where(mask)[0]}")

Removed 10 features with numbers [ 32  33  35  40  62  70  73  97 124 148]


In [127]:
del_idx = np.where(mask)[0]
new_features = []
for i in range(len(features)):
    if i not in del_idx:
        new_features.append(features[i])

In [128]:
sub_train_cnt = np.sum(X_train_selected, axis=1).reshape(-1, 1)
sub_eval_cnt = np.sum(X_eval_selected, axis=1).reshape(-1, 1)
sub_test_cnt = np.sum(X_test_selected, axis=1).reshape(-1, 1)

X_train_with_subs = np.hstack((X_train_selected, sub_train_cnt))
X_eval_with_subs = np.hstack((X_eval_selected, sub_eval_cnt))
X_test_with_subs = np.hstack((X_test_selected, sub_test_cnt))
new_features.append("subscription_cnt")

In [129]:
max_depth = range(1, 10, 3)
min_samples_leaf = range(1, 10, 3)
learning_rate = np.linspace(0.001, 1.0, 5)


def get_best_params(y_train, y_eval):
    best_score, best_params = None, None
    for lr, md, msl in tqdm(list(product(learning_rate, max_depth, min_samples_leaf))):
        cb_model = CatBoostClassifier(
            loss_function="MultiClass", verbose=0, learning_rate=lr, max_depth=md, min_data_in_leaf=msl
        )
        cb_model.fit(X_train_with_subs, y_train)
        y_pred = cb_model.predict(X_eval_with_subs)
        score = accuracy_score(y_eval, y_pred)
        if best_score is None or score > best_score:
            best_score = score
            best_params = {"learning_rate": lr, "max_depth": md, "min_data_in_leaf": msl}

    return best_params, best_score

#### Возраст

In [130]:
best_params, best_score = get_best_params(y_age_train_unique, y_age_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075), 'max_depth': 1, 'min_data_in_leaf': 1},
 0.7554309740714786)

In [131]:
cb_model = CatBoostClassifier(loss_function="MultiClass", verbose=0, **best_params)
cb_model.fit(X_train_with_subs, y_age_train_unique)
y_pred = cb_model.predict(X_test_with_subs)

print("Accuracy:", accuracy_score(y_age_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.725094577553594
Most important features:
1. privetuyeba
2. styd.pozor
3. chop.choppp
4. videosos
5. rhymes
6. cook_bon
7. onlyorly
8. gm.power
9. tumblr_vacuum
10. ne.poverish


#### Пол

In [133]:
best_params, best_score = get_best_params(y_sex_train_unique, y_sex_eval)
best_params, best_score

  0%|          | 0/45 [00:00<?, ?it/s]

({'learning_rate': np.float64(0.25075), 'max_depth': 4, 'min_data_in_leaf': 1},
 0.8689558514365803)

In [134]:
cb_model = CatBoostClassifier(loss_function="MultiClass", verbose=0, **best_params)
cb_model.fit(X_train_with_subs, y_sex_train_unique)
y_pred = cb_model.predict(X_test_with_subs)

print("Accuracy:", accuracy_score(y_sex_test, y_pred))
print("Most important features:")
for i, name in enumerate(most_important_features(cb_model.feature_importances_, features, 10)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8638083228247163
Most important features:
1. mash
2. 40kg
3. gm.power
4. club52205838
5. girlmeme
6. 9o_6o_9o
7. rapnewrap
8. nenormalnoo
9. exclusive_muzic
10. ne.poverish
